In [1]:
import pygsheets
import pandas as pd
import datetime
import json
import random
import numpy as np
from IPython.core import display as ICD
from IPython.display import clear_output
pd.set_option('display.max_rows', 100)

gc=pygsheets.authorize(service_file='camelotspreadsheets-5fdda9ce433a.json')

In [2]:
spreadsheet_name = "base_dishes"
sh = gc.open(spreadsheet_name)

In [3]:
alltipos_salgados = ['proteina','base','mistura']

In [4]:
sh = gc.open(spreadsheet_name)
wks = sh.worksheet('title','salgados')

pratos_salgados = wks.get_as_df()

In [5]:
pratos_salgados['tipos'] = [t.split('&') for t in pratos_salgados['tipos']]
pratos_salgados['ingredientes'] = [json.loads(i.replace("'",'"')) for i in pratos_salgados['ingredientes']]

In [6]:
def monta_um_prato(df_pratos,alltipos):
    df_pratos=df_pratos.copy()
    # Pega um prato aleatório do set de pratos
    prato_now = df_pratos.copy().sample()
#     print("Selecionei o primeiro prato:")
#     ICD.display(prato_now)
    tipos_contemplados = prato_now['tipos'].values[0].copy()
    # Se o prato selecionado for do tipo 'unico', ja retorna o prato
    if 'unico' in tipos_contemplados: 
        prato_now['multiplicador_receitas'] = 1
        return prato_now
    else:
        # Começa a montar a composição de pratos para ter alltipos na refeição
#         print("Comecei um prato composto:")
        prato_composto = prato_now.copy()
#         ICD.display(prato_composto)
        tipos_que_faltam = list(set(alltipos)-set(tipos_contemplados))
#         tipos_now=tipos_que_faltam.copy()
        # Enquanto não possuir todos os tipos de alltipos, busca novos pratos
        while set(tipos_contemplados)!=set(alltipos):
            # Pega apenas pratos que possuem tipos complementares aos tipos já selecionados
            pratos_possiveis = df_pratos.loc[[len(list(set(t) & set(tipos_contemplados+['unico'])))==0 for t in df_pratos['tipos']]].copy()
            # Filtra apenas pratos cuja porção seja multipla inteira da máxima porção já selecionada
            max_porcao = prato_composto['porcoes'].max()
            porcoes_multiplas = [n for n in range(1,max_porcao+1) if (max_porcao/n)%np.floor(max_porcao/n)==0]
            pratos_possiveis = pratos_possiveis.loc[[p in porcoes_multiplas for p in pratos_possiveis['porcoes']]]
            # Se não foi possível montar um prato, chama a função novamente
            if pratos_possiveis.shape[0] == 0: 
                return monta_um_prato(df_pratos,alltipos)
#             print("selecionei um novo prato: ")
            prato_now = pratos_possiveis.copy().sample()
#             ICD.display(prato_now)
            prato_composto = prato_composto.append(prato_now)
#             print("Agora o prato composto é: ")
#             ICD.display(prato_composto)
            tipos_contemplados+=prato_now['tipos'].values[0].copy()
        prato_composto['multiplicador_receitas'] = [prato_composto['porcoes'].max()/p for p in prato_composto['porcoes']]
        return prato_composto
    
def display_refeicoes(lista):
    for r in lista: ICD.display(r)
        
def gera_lista_de_compras(refeicoes_list):
    all_listas = [x for l in [df['ingredientes'].values for df in refeicoes_list] for x in l]
    multiplicadores = [x for l in [df['multiplicador_receitas'].values for df in refeicoes_list] for x in l]
    lista_geral = pd.DataFrame(columns=['item','qtd','unid'])
    for (mult,lista) in zip(multiplicadores,all_listas):
        for itm,ifo in lista.items():
            # Se o item já está na lista, com a mesma unidade
            if lista_geral.loc[(lista_geral['item']==itm)&(lista_geral['unid']==ifo.split("&")[1])].shape[0] > 0:
                # Soma as quantidades
                lista_geral.loc[(lista_geral['item']==itm)&(lista_geral['unid']==ifo.split("&")[1]),'qtd']+=(float(ifo.split("&")[0])*mult)
            # Se não, adiciona item na lista
            else:
                lista_geral=lista_geral.append({'item':itm,
                                                'qtd':float(ifo.split("&")[0])*mult,
                                                'unid':ifo.split("&")[1]
                                               },ignore_index=True)
    return lista_geral.sort_values(by='item').reset_index(drop=True)

def gera_calendario_de_pratos(refeicoes_list,pessoas,refeicoes_por_dia):
    dia_now = 1
    df_calendario = pd.DataFrame(columns=['Dia','Pratos'])
    for df in refeicoes_list:
        dias_receita = df['porcoes'].max()/(pessoas*refeicoes_por_dia)
        df_tmp = df[['nome']].copy().rename(columns={'nome':'Pratos'})
        for d in range(int(np.floor(dia_now)),int(np.ceil(dias_receita))+int(np.floor(dia_now))):
            df_tmp['Dia'] = d
            df_calendario=df_calendario.append(df_tmp[['Dia','Pratos']])
        dia_now+=dias_receita
    return df_calendario.reset_index(drop=True)

def df_to_sheet(df,header=True):
    df_columns = [np.array(df.columns)]
    df_values = df.values.tolist()
    df_to_sheet = np.concatenate((df_columns, df_values)).tolist()
    if header: return df_to_sheet
    else: return df_values

### Planejamento ao longo de D dias

In [7]:
pessoas = 2
refeicoes_por_dia = 2
dias_a_planejar = 5
porcoes_extra = 4

In [8]:
porcoes_totais = pessoas*refeicoes_por_dia*dias_a_planejar
refeicoes_list = []
porcoes_now = 0
dias_calculados = 0
while porcoes_now<porcoes_totais:
    # Considera apenas pratos cujos ingredientes durem até a data de prepará-los e cuja porção não ultrapasse em demasia o calculo
    pratos_viaveis = pratos_salgados.loc[(pratos_salgados['durab_ingredientes']>=dias_calculados)&(pratos_salgados['porcoes']<=(porcoes_totais-porcoes_now)+porcoes_extra)].copy()
    # Sugere uma refeição para o usuário
    refeicao_now = monta_um_prato(pratos_viaveis,alltipos_salgados)
    resp = "X"
    while resp.lower() not in ["s","n"]:
        clear_output()
        ICD.display(refeicao_now)
        resp = input("Você gostou dessa refeição (S/N)? ")
    # Se o usuário gostou, adiciona refeição na lista de refeições
    if resp.lower()=='s':
        refeicoes_list+=[refeicao_now]
        porcoes_now+=refeicao_now['porcoes'].max()
        dias_calculados+=refeicao_now['porcoes'].max()/(pessoas*refeicoes_por_dia)
clear_output()
print("Pratos selecionados:")
display_refeicoes(refeicoes_list)
compras_list = gera_lista_de_compras(refeicoes_list)
ICD.display(compras_list)

Pratos selecionados:


,nome,tipos,porcoes,durab_ingredientes,durab_alimento,congelavel,complexidade,clima,ingredientes,link_receita,dica,multiplicador_receitas
23,panquecas mistas,"[proteina, mistura]",12,5,5,TRUE,4,ameno,"{'ovo': '3&unid', 'leite': '750&ml', 'farinha'...",https://www.tudogostoso.com.br/receita/88473-p...,ver receita da Janaina. pode fazer as massas a...,1.0
25,pure de batata,[base],6,15,6,FALSE,1,ameno&frio,"{'batata': '5&unid', 'leite': '150&ml', 'marga...",https://www.tudogostoso.com.br/receita/59-pure...,O segredo é bater bem,2.0


,nome,tipos,porcoes,durab_ingredientes,durab_alimento,congelavel,complexidade,clima,ingredientes,link_receita,dica,multiplicador_receitas
19,macarrao a carbonara,[unico],6,20,6,FALSE,2,ameno,"{'espaguete': '1&pacote', 'bacon': '1&pacote',...",https://www.tudogostoso.com.br/receita/1624-ma...,None,1


,nome,tipos,porcoes,durab_ingredientes,durab_alimento,congelavel,complexidade,clima,ingredientes,link_receita,dica,multiplicador_receitas
25,pure de batata,[base],6,15,6,FALSE,1,ameno&frio,"{'batata': '5&unid', 'leite': '150&ml', 'marga...",https://www.tudogostoso.com.br/receita/59-pure...,O segredo é bater bem,1.0
9,carne de panela com cenoura,"[proteina, mistura]",6,5,5,TRUE,2,ameno&frio,"{'coxao mole': '600&g', 'cenoura': '2&unid', '...",https://www.tudogostoso.com.br/receita/8836-ca...,None,1.0


,item,qtd,unid
0,alho,4.0,col_sopa
1,bacon,1.0,pacote
2,batata,15.0,unid
3,calabresa fina,1.0,pacote
4,caldo de galinha,1.0,tablete
5,carne moida,500.0,g
6,cebola,2.0,unid
7,cenoura,2.0,unid
8,cheiro verde,0.5,maco
9,coxao mole,600.0,g


In [9]:
# Atualiza sheet da planilha cardapio_automatico
wks_output_plano = gc.open('cardapio_automatico').worksheet('title','planejamento')
wks_output_calend = gc.open('cardapio_automatico').worksheet('title','calendario')
wks_output_compras = gc.open('cardapio_automatico').worksheet('title','lista de compras')

# Insere pratos planejados
df_allrecipes = pd.DataFrame()
for df in refeicoes_list:
    df_allrecipes=df_allrecipes.append(df)
wks_output_plano.clear()
df_allrecipes_tmp = df_allrecipes[['nome','porcoes','multiplicador_receitas','durab_alimento','congelavel','complexidade','dica','link_receita','ingredientes']].copy()
df_allrecipes_tmp['ingredientes'] = ["{}".format(i) for i in df_allrecipes_tmp['ingredientes']]
wks_output_plano.update_values("A1",df_to_sheet(df_allrecipes_tmp))

# Insere calendário de pratos
df_calendario = gera_calendario_de_pratos(refeicoes_list,pessoas,refeicoes_por_dia)
wks_output_calend.clear()
s_row = 1
for d in sorted(np.unique(df_calendario['Dia'])):
    df_c_tmp = df_calendario.loc[df_calendario['Dia']==d,['Pratos']]
    wks_output_calend.update_value("A%d"%(s_row),"Dia %d:"%d)
    wks_output_calend.update_values("A%d"%(s_row+1),df_to_sheet(df_c_tmp,header=False))
    s_row+=(df_c_tmp.shape[0]+2)

# Insere lista de compras
df_compras = compras_list[['item']].copy()
df_compras['qtd'] = compras_list[['qtd','unid']].apply(lambda r: "%.1f %s"%(r['qtd'],r['unid']),axis=1)
wks_output_compras.clear()
wks_output_compras.update_values("A1",df_to_sheet(df_compras))